# Multi-Head Attention Minibenchmark
This benchmark is taken from [this Pytorch tutorial](https://pytorch.org/tutorials/prototype/nestedtensor.html).

This benchmark expects packages from the `requirements.txt` in the root directory to be installed.

In [9]:
import numpy as np
import timeit
import torch
import torch.nn.functional as F
import tqdm

from torch import nn

torch.manual_seed(1)
np.random.seed(1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
class MultiHeadAttention(nn.Module):
    """
    Computes multi-head attention. Supports nested or padded tensors.

    Args:
        E_q (int): Size of embedding dim for query
        E_k (int): Size of embedding dim for key
        E_v (int): Size of embedding dim for value
        E_total (int): Total embedding dim of combined heads post input projection. Each head
            has dim E_total // nheads
        nheads (int): Number of heads
        dropout_p (float, optional): Dropout probability. Default: 0.0
    """
    def __init__(self, E_q: int, E_k: int, E_v: int, E_total: int,
                 nheads: int):
        super().__init__()
        self.nheads = nheads
        self.query_proj = nn.Linear(E_q, E_total)
        self.key_proj = nn.Linear(E_k, E_total)
        self.value_proj = nn.Linear(E_v, E_total)
        E_out = E_q
        self.out_proj = nn.Linear(E_total, E_out)
        assert E_total % nheads == 0, "Embedding dim is not divisible by nheads"
        self.E_head = E_total // nheads

    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor) -> torch.Tensor:
        """
        Forward pass; runs the following process:
            1. Apply input projection
            2. Split heads and prepare for SDPA
            3. Run SDPA
            4. Apply output projection

        Args:
            query (torch.Tensor): query of shape (N, L_t, E_q)
            key (torch.Tensor): key of shape (N, L_s, E_k)
            value (torch.Tensor): value of shape (N, L_s, E_v)

        Returns:
            attn_output (torch.Tensor): output of shape (N, L_t, E_q)
        """
        # Step 1. Apply input projection
        # TODO: demonstrate packed projection
        query = self.query_proj(query)
        key = self.key_proj(key)
        value = self.value_proj(value)

        # Step 2. Split heads and prepare for SDPA
        # reshape query, key, value to separate by head
        # (N, L_t, E_total) -> (N, L_t, nheads, E_head) -> (N, nheads, L_t, E_head)
        query = query.unflatten(-1, [self.nheads, self.E_head]).transpose(1, 2)
        # (N, L_s, E_total) -> (N, L_s, nheads, E_head) -> (N, nheads, L_s, E_head)
        key = key.unflatten(-1, [self.nheads, self.E_head]).transpose(1, 2)
        # (N, L_s, E_total) -> (N, L_s, nheads, E_head) -> (N, nheads, L_s, E_head)
        value = value.unflatten(-1, [self.nheads, self.E_head]).transpose(1, 2)

        # Step 3. Run SDPA
        # (N, nheads, L_t, E_head)
        attn_output = F.scaled_dot_product_attention(
            query, key, value, is_causal=True)
        # (N, nheads, L_t, E_head) -> (N, L_t, nheads, E_head) -> (N, L_t, E_total)
        attn_output = attn_output.transpose(1, 2).flatten(-2)

        # Step 4. Apply output projection
        # (N, L_t, E_total) -> (N, L_t, E_out)
        attn_output = self.out_proj(attn_output)

        return attn_output

In [11]:
N = 512
E_q, E_k, E_v, E_total = 512, 512, 512, 512
E_out = E_q
nheads = 8

In [12]:
def zipf_sentence_lengths(alpha: float, batch_size: int) -> torch.Tensor:
    # generate fake corpus by unigram Zipf distribution
    # from wikitext-2 corpus, we get rank "." = 3, "!" = 386, "?" = 858
    sentence_lengths = np.empty(batch_size, dtype=int)
    for ibatch in range(batch_size):
        sentence_lengths[ibatch] = 1
        word = np.random.zipf(alpha)
        while word != 3 and word != 386 and word != 858:
            sentence_lengths[ibatch] += 1
            word = np.random.zipf(alpha)
    return torch.tensor(sentence_lengths)

In [21]:
def gen_batch(N, E_q, E_k, E_v, device):
    # generate semi-realistic data using Zipf distribution for sentence lengths
    sentence_lengths = zipf_sentence_lengths(alpha=1.2, batch_size=N)

    # Note: the torch.jagged layout is a nested tensor layout that supports a single ragged
    # dimension and works with torch.compile. The batch items each have shape (B, S*, D)
    # where B = batch size, S* = ragged sequence length, and D = embedding dimension.
    query = torch.nested.nested_tensor([
        torch.randn(l.item(), E_q, device=device)
        for l in sentence_lengths
    ], layout=torch.jagged).to(device=device)

    key = torch.nested.nested_tensor([
        torch.randn(s.item(), E_k, device=device)
        for s in sentence_lengths
    ], layout=torch.jagged).to(device=device)

    value = torch.nested.nested_tensor([
        torch.randn(s.item(), E_v, device=device)
        for s in sentence_lengths
    ], layout=torch.jagged).to(device=device)

    return query, key, value, sentence_lengths

query, key, value, sentence_lengths = gen_batch(N, E_q, E_k, E_v, device)

In [22]:
def jagged_to_padded(jt, padding_val):
    # TODO: do jagged -> padded directly when this is supported
    return torch.nested.to_padded_tensor(
        torch.nested.nested_tensor(list(jt.unbind())),
        padding_val).to(device=device)

padded_query, padded_key, padded_value = (
    jagged_to_padded(t, 0.0) for t in (query, key, value)
)

In [23]:
mha = MultiHeadAttention(E_q, E_k, E_v, E_total, nheads).to(device=device)

## Timing MHA Runtime

In [24]:
def benchmark(func, *args, **kwargs):
    torch.cuda.synchronize()
    begin = timeit.default_timer()
    output = func(*args, **kwargs)
    torch.cuda.synchronize()
    end = timeit.default_timer()
    return output, (end - begin)

num_trials = 100

compiled_time_nested = []
compiled_time_padded = []

# warm up compile first...
compiled_mha = torch.compile(
            mha, 
            #options={"triton.cudagraphs": True}, 
            fullgraph=True,
            backend='cudagraphs'
    )

compiled_mha(query, key, value)

# ...now benchmark
for _ in tqdm.trange(1, num_trials + 1):
    _, t = benchmark(compiled_mha, query, key, value)
    compiled_time_nested.append(t)

print("Average ragged tensor runtime:", sum(compiled_time_nested) / len(compiled_time_nested), "seconds")

torch.compiler.reset()

# warm up compile first...
compiled_mha = torch.compile(
            mha, 
            #options={"triton.cudagraphs": True}, 
            fullgraph=True,
            backend='cudagraphs'
    )

# warm up compile first...
compiled_mha(padded_query, padded_key, padded_value)
# ...now benchmark

for _ in tqdm.trange(1, num_trials + 1):
    _, t = benchmark(compiled_mha, padded_query, padded_key, padded_value)
    compiled_time_padded.append(t)

print("Average padded tensor runtime:", sum(compiled_time_padded) / len(compiled_time_padded), "seconds")

skipping cudagraphs due to skipping cudagraphs due to cpu device (zeros)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 590.50it/s]


Average ragged tensor runtime: 0.0016675735265016555 seconds


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 98.57it/s]

Average padded tensor runtime: 0.010072691394016146 seconds


## Timing MHA JIT

In [19]:
def benchmark_jit(func, *args, **kwargs):
    torch.compiler.reset()
    torch.cuda.synchronize()
    begin = timeit.default_timer()
    compiled_func = torch.compile(
                func, 
                #options={"triton.cudagraphs": True}, 
                fullgraph=True,
                backend='cudagraphs'
        )
    output = compiled_func(*args, **kwargs)
    torch.cuda.synchronize()
    end = timeit.default_timer()
    return output, (end - begin)

num_jit_trials = 10

jit_time_nested = []
jit_time_padded = []

for _ in tqdm.trange(1, num_jit_trials + 1):
    _, t = benchmark_jit(mha, query, key, value)
    jit_time_nested.append(t)

print("Average ragged tensor JIT time:", sum(jit_time_nested) / len(jit_time_nested), "seconds")

for _ in tqdm.trange(1, num_trials + 1):
    _, t = benchmark_jit(mha, padded_query, padded_key, padded_value)
    jit_time_padded.append(t)

print("Average padded tensor JIT time:", sum(jit_time_padded) / len(jit_time_padded), "seconds")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.77it/s]


Average ragged tensor JIT time: 0.26472784793004395 seconds


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:15<00:00,  6.58it/s]

Average padded tensor JIT time: 0.15139416206628084 seconds
